# Week 4 - Models and Experimentation

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

### Install packages

In [1]:
# !pip install wandb -qU
# already installed

In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [3]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [5]:
diamonds.shape

(53940, 10)

In [6]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

### Split the data and train a model

In [7]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [8]:
# Define hyperparameters
# params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
params = {"objective": "reg:squarederror"} # Training on my local machine, no GPU

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

In [9]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 545.191877397669


### Incorporate validation

In [10]:
# params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
params = {"objective": "reg:squarederror"} # Training on my local machine, no GPU

n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [11]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2861.71326	validation-rmse:2853.85688
[10]	train-rmse:554.29819	validation-rmse:579.26422
[20]	train-rmse:493.68077	validation-rmse:547.75493
[30]	train-rmse:467.32713	validation-rmse:540.03567
[40]	train-rmse:447.40974	validation-rmse:541.70531
[50]	train-rmse:432.62075	validation-rmse:540.89769
[60]	train-rmse:422.28318	validation-rmse:540.63039
[70]	train-rmse:410.72350	validation-rmse:543.67077
[80]	train-rmse:398.24619	validation-rmse:545.08296
[90]	train-rmse:386.92486	validation-rmse:543.90036
[99]	train-rmse:379.58717	validation-rmse:545.19188


In [12]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2861.71326	validation-rmse:2853.85688
[50]	train-rmse:432.62075	validation-rmse:540.89769
[82]	train-rmse:394.92609	validation-rmse:544.73937


In [13]:
# Cross-validation

# params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
params = {"objective": "reg:squarederror"} # Training on my local machine, no GPU
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


In [14]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.512810,8.494816,2861.704341,37.144992
1,2081.847733,5.811005,2084.838207,31.889208
2,1547.031906,5.092391,1554.657450,30.699908
3,1184.129738,3.982239,1194.251600,26.940062
4,942.998782,3.327174,960.239319,24.392689


In [15]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

553.4613038243663

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [16]:
!env WANDB_API_KEY=$c82395540f09bcf90536f9216e8c1eb4f1b070ac

COMMAND_MODE=unix2003
CONDA_DEFAULT_ENV=prac
CONDA_EXE=/Users/yash/anaconda3/bin/conda
CONDA_PREFIX=/Users/yash/anaconda3/envs/prac
CONDA_PROMPT_MODIFIER=(prac) 
CONDA_PYTHON_EXE=/Users/yash/anaconda3/bin/python
CONDA_SHLVL=2
HOME=/Users/yash
HOMEBREW_CELLAR=/opt/homebrew/Cellar
HOMEBREW_PREFIX=/opt/homebrew
HOMEBREW_REPOSITORY=/opt/homebrew
INFOPATH=/opt/homebrew/share/info:
LOGNAME=yash
MANPATH=/opt/homebrew/share/man::
MallocNanoZone=0
OLDPWD=/Users/yash/Downloads
ORIGINAL_XDG_CURRENT_DESKTOP=undefined
PATH=/Users/yash/anaconda3/envs/prac/bin:/Users/yash/anaconda3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Library/Apple/usr/bin
PWD=/Users/yash/Downloads
SHELL=/bin/zsh
SHLVL=2
SSH_AUTH_

In [17]:
# Log in to your W&B account
import wandb

wandb.login() # c82395540f09bcf90536f9216e8c1eb4f1b070ac

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashkhurana2024. Use `wandb login --relogin` to force relogin


True

In [18]:
# TO DO
# Start experiment tracking with W&B
# Do at least 5 experiments with various hyperparameters
# Choose any method for hyperparameter tuning: grid search, random search, bayesian search
# Describe your findings and what you see

#### Experiment - 1

In [19]:
sweep_config = {
    "method": "random",
    "metric": {
      "name": "rmse",
      "goal": "minimize"   
    },
    "parameters": {
        'max_depth': {
            'values': [3, 5, 7, 9]
        },
        'subsample': {
            'min': 0.6,
            'max': 0.9
        },
        'colsample_bytree': {
            'min': 0.6,
            'max': 0.9
        },
        'n_estimators': {
            'values': [50, 100, 150, 200]
        }
    }
}

In [20]:
sweep_id = wandb.sweep(sweep_config, project="experiment-1")

Create sweep with ID: qz8ivpt5
Sweep URL: https://wandb.ai/yashkhurana2024/experiment-1/sweeps/qz8ivpt5


In [21]:
def train():
    config_defaults = {
        "objective": "reg:squarederror"
    }

    wandb.init(config=config_defaults)
    config = wandb.config

    n = 10000

    model = xgb.train(
        params=config_defaults,
        dtrain=dtrain,
        num_boost_round=n,
        evals=evals,
        verbose_eval=50,
        # Activate early stopping
        early_stopping_rounds=10
    )
    
    predictions = model.predict(dtest)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f"RMSE: {rmse}")
    wandb.log({"rmse": rmse})

In [22]:
wandb.agent(sweep_id, train, count=25)

wandb: Agent Starting Run: nw89i3wa with config:
wandb: 	colsample_bytree: 0.824976548262328
wandb: 	max_depth: 9
wandb: 	n_estimators: 150
wandb: 	subsample: 0.6763973385078461
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


[0]	train-rmse:2861.71326	validation-rmse:2853.85688
[43]	train-rmse:444.26578	validation-rmse:541.93113
RMSE: 541.9311265694964


rmse,▁
rmse,541.93113


wandb: Agent Starting Run: 9ae7lihz with config:
wandb: 	colsample_bytree: 0.7017698717712626
wandb: 	max_depth: 5
wandb: 	n_estimators: 200
wandb: 	subsample: 0.7533948921282512
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


[0]	train-rmse:2861.71326	validation-rmse:2853.85688
[42]	train-rmse:446.52109	validation-rmse:541.38770
RMSE: 541.9311265694964


...

[0]	train-rmse:2861.71326	validation-rmse:2853.85688
[42]	train-rmse:446.52109	validation-rmse:541.38770
RMSE: 541.9311265694964


rmse,▁
rmse,541.93113


wandb: Agent Starting Run: o6nugud2 with config:
wandb: 	colsample_bytree: 0.8954377691972203
wandb: 	max_depth: 3
wandb: 	n_estimators: 100
wandb: 	subsample: 0.7211441398864381
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


[0]	train-rmse:2861.71326	validation-rmse:2853.85688
[43]	train-rmse:444.26578	validation-rmse:541.93113
RMSE: 541.9311265694964


rmse,▁
rmse,541.93113


### Findings

- The **colsample_bytree** parameter, which controls the fraction of features to be randomly sampled for each tree, was varied between approximately 0.62 and 0.89. The best RMSE was achieved with a value of 541.9311265694964 at 0.62. 
- The **max_depth** parameter, which controls the maximum depth of the tree, was varied between 3 and 9. The best RMSE was achieved with a value of 541.9311265694964 at 9.
- The **n_estimators** parameter, which controls the number of boosting rounds, was varied between 50 and 200. The best RMSE was achieved with a value of 541.9311265694964 at 200.
- The **subsample** parameter, which controls the fraction of samples to be randomly sampled for each tree, was varied between approximately 0.62 and 0.89. The best RMSE was achieved with a value of 541.9311265694964 at 0.62.

The best RMSE was achieved with a value of 541.9311265694964 at 0.62. This indicates that the model is able to predict the price of diamonds with an average error of approximately $541.93. The hyperparameters that achieved this result were a colsample_bytree value of 0.62, a max_depth value of 9, a n_estimators value of 200, and a subsample value of 0.62. These hyperparameters were found using random search with 25 trials.